In [19]:
import torch
from copy import deepcopy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [20]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("SetFit/subj")
dataset = dataset.remove_columns("label_text")
dataset = dataset['train'].shuffle(seed=42).select([i for i in list(range(6000))])
train_ds = Dataset.from_dict(dataset[0:5000])
validation_ds = Dataset.from_dict(dataset[5000:5500])
test_ds = Dataset.from_dict(dataset[5500:6000])

data = test_ds





Using custom data configuration SetFit___subj-848ec40cb3887b72
Reusing dataset json (C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)
100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.22it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-80b8e4129e527f71.arrow


In [21]:
display(data)
display(data[0:5])

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

{'text': ['he can remember nothing and begins to try to rebuild his memory based on clues such as the swiss bank account , the number of which , is implanted in his hip .',
  'leaving her successful career and las vegas forever in order to build her family in a healthy environment was since long prevented by katherine hiller ( barbara carrera ) , her discoverer and business manager .',
  "a film worthy of comparison to truffaut 's best cinematic poems .",
  'in the end there is one word that best describes this film : honest .',
  'angry and confused teen runaways come and go at the shelter , as steve learns he is not alone in his predicament and that the streets offer a harsh reality .'],
 'label': [0, 0, 1, 1, 0]}

In [22]:
import textattack
dataset = textattack.datasets.HuggingFaceDataset(data)
display(dataset)

In [23]:
#attack the original model using TextFooler
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model_path = r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\CheckList\run3'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)



In [24]:
from textattack.attack_recipes.pwws_ren_2019 import PWWSRen2019
attack = PWWSRen2019.build(model_wrapper)
attack.cuda_()
display(attack)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)

In [25]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=-1, random_seed = 765,
                         log_to_csv = r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\results\CheckList\run3\pwws_attack_results.csv', 
                         csv_coloring_style = 'file')

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


textattack: Logging to CSV at path C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\results\CheckList\run3\pwws_attack_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



  0%|▏                                                                                 | 1/500 [00:00<08:10,  1.02it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 1/500 [00:00<08:17,  1.00it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (99%)]] --> [[1 (87%)]]

he can remember [[nothing]] and [[begins]] to [[try]] to rebuild his [[memory]] based on [[clues]] such as the swiss bank account , the number of which , is [[implanted]] in his hip .

he can remember [[zip]] and [[commence]] to [[taste]] to rebuild his [[retentiveness]] based on [[hint]] such as the swiss bank account , the number of which , is [[engraft]] in his hip .




[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 2/500 [00:02<09:03,  1.09s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                            | 2/500 [00:02<09:08,  1.10s/it]

...

--------------------------------------------- Result 498 ---------------------------------------------
[[0 (99%)]] --> [[1 (63%)]]

maggie is a [[young]] cuban-american [[freelance]] writer who [[takes]] an eye opening [[trip]] back to her [[roots]] and [[meets]] a [[young]] enchanting native , mateo .

maggie is a [[unseasoned]] cuban-american [[independent]] writer who [[convey]] an eye opening [[misstep]] back to her [[theme]] and [[fit]] a [[untested]] enchanting native , mateo .




C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 308 / 165 / 26 / 499: 100%|██████████████████▉| 499/500 [06:52<00:00,  1.21it/s]

--------------------------------------------- Result 499 ---------------------------------------------
[[0 (99%)]] --> [[1 (95%)]]

[[one]] stormy [[night]] , they [[go]] to [[get]] the shoes when calvin and the shoes are struck by lightning .

[[I]] stormy [[Nox]] , they [[fit]] to [[dumbfound]] the shoes when calvin and the shoes are struck by lightning .




[Succeeded / Failed / Skipped / Total] 308 / 165 / 26 / 499: 100%|███████████████████| 500/500 [06:53<00:00,  1.21it/s]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 309 / 165 / 26 / 500: 100%|███████████████████| 500/500 [06:53<00:00,  1.21it/s]

--------------------------------------------- Result 500 ---------------------------------------------
[[0 (95%)]] --> [[1 (98%)]]

the [[girls]] have unwittingly botched up a diamond delivery to the middle east .

the [[fille]] have unwittingly botched up a diamond delivery to the middle east .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 309    |
| Number of failed attacks:     | 165    |
| Number of skipped attacks:    | 26     |
| Original accuracy:            | 94.8%  |
| Accuracy under attack:        | 33.0%  |
| Attack success rate:          | 65.19% |
| Average perturbed word %:     | 18.37% |
| Average num. words per input: | 21.97  |
| Avg num queries:              | 195.97 |
+-------------------------------+--------+


In [26]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))
results_html = logger.df[['original_text', 'perturbed_text']].to_html(escape=False)
path = r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\results\CheckList\run3\pwws_attack_results.html'
text_file = open(path, "w")
text_file.write(results_html.encode("gbk", 'ignore').decode("gbk", "ignore"))
text_file.close()
logger.flush()

textattack: Logging to CSV at path results.csv
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated 

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,original_text,perturbed_text
0,"he can remember nothing and begins to try to rebuild his memory based on clues such as the swiss bank account , the number of which , is implanted in his hip .","he can remember zip and commence to taste to rebuild his retentiveness based on hint such as the swiss bank account , the number of which , is engraft in his hip ."
1,"leaving her successful career and las vegas forever in order to build her family in a healthy environment was since long prevented by katherine hiller ( barbara carrera ) , her discoverer and business manager .","allow her successful career and las vegas forever in rescript to build her category in a healthy environment was since recollective forestall by katherine hiller ( barbara carrera ) , her discoverer and byplay manager ."
2,a film worthy of comparison to truffaut 's best cinematic poems .,a celluloid suitable of compare to truffaut 's trump cinematic poems .
3,in the end there is one word that best describes this film : honest .,in the end there is one Bible that practiced describes this take : true .
4,"angry and confused teen runaways come and go at the shelter , as steve learns he is not alone in his predicament and that the streets offer a harsh reality .","angry and confused teen laugher fare and go at the shelter , as steve ascertain he is not alone in his predicament and that the streets offer a harsh realism ."
5,except shaun runs into one complication after another starting when his application is rejected after his dim-witted guidance counselor sends the wrong application .,except shaun streamlet into one complication after another starting when his coating is rejected after his dim-witted guidance counselor sends the wrong application .
6,heart problems force boston cop pally lamarr to retire ; he 's in a funk and his wife charlotte sends him packing .,warmheartedness problems forcefulness boston cop pally lamarr to retire ; he 's in a funk and his wife charlotte direct him packing .
7,"maggie 's father , yayo , runs a successful bakery , steve is a politician , elena is a fashion designer , ms . al seeks love , conrado is a bashful deliveryman , and mateo is looking for a chance in life .","maggie 's beget , yayo , runs a successful bakehouse , steve is a pol , elena is a way designer , ms . al attempt enjoy , conrado is a bashful deliveryman , and mateo is expect for a chance in animation ."
8,a high-spirited buddy movie about the reunion of berlin anarchists who face arrest 15 years after their crime .,a high-spirited buddy movie about the reunion of berlin anarchists who face arrest 15 years after their crime .
9,"even with harris 's strong effort , the script gives him little to effectively probe lear 's soul-stripping breakdown .","eve with harris 's strong cause , the book gives him picayune to effectively probe lear 's soul-stripping breakdown ."


In [27]:
import textattack

attack_log_manager = textattack.loggers.AttackLogManager()


for result in attack_results:
    attack_log_manager.log_result(result)

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()

attack_log_manager.flush()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 309    |
| Number of failed attacks:     | 165    |
| Number of skipped attacks:    | 26     |
| Original accuracy:            | 94.8%  |
| Accuracy under attack:        | 33.0%  |
| Attack success rate:          | 65.19% |
| Average perturbed word %:     | 18.37% |
| Average num. words per input: | 21.97  |
| Avg num queries:              | 195.97 |
+-------------------------------+--------+